In [ ]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
class myDS(Dataset):
    def __init__(self, data, in_cols, cond_cols):
        self.data = data
        self.in_cols = in_cols
        self.cond_cols = cond_cols
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        out = self.data.iloc[idx]
        return out[self.in_cols].values, out[self.cond_cols].values
        

In [ ]:
SEED = 42
BATCH_SIZE = 256
EPOCHS = 5
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train, test = train_test_split(df_kaon)
train_loader = DataLoader(myDS(train,y_cols, x_cols), batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(myDS(test,y_cols, x_cols), batch_size = BATCH_SIZE, shuffle = True)

input_dim = len(y_cols)
latent_dim =  2
cond_dim = len(x_cols)

from models import CVAE
from torch.autograd import Variable

encoder_dims = [input_dim + cond_dim, 8,16,8]
decoder_dims = [8, 16,8, input_dim]



from importlib import reload 
CVAE = reload(CVAE)

model = CVAE(encoder_dims, latent_dim, decoder_dims, cond_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, cond) in enumerate(train_loader):
        
        data = Variable(data.to(device))
        cond = Variable(cond.to(device))
        
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data,cond)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data,cond) in enumerate(test_loader):
            data = Variable(data.to(device))
            cond = Variable(cond.to(device))
            recon_batch, mu, logvar = model(data,cond)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')
